<a href="https://colab.research.google.com/github/tomhyhan/ML/blob/main/ML/colab/food.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
import torch as t
import torchvision
import torchvision.transforms as tt
from torch.utils.data import Subset, random_split

In [17]:
def load_data():
    transform = tt.Compose([
        tt.Resize(56),
        tt.CenterCrop(28),
        tt.ToTensor(),
        tt.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    food101_data =torchvision.datasets.Food101(download=True, root="./", transform=transform)
    print("transform done")
    subset_indices = set()
    subset_images = []
    for i in range(len(food101_data)):
        if i % 1000 == 0:
            print("iter: ", i)
            print(subset_indices)
        subset_indices.add(food101_data[i][1])
        subset_images.append(i)
        if len(subset_indices) > 20:
            break
    food101_subset = Subset(food101_data, subset_images)
    print("len: ", len(food101_subset))
    traning_i = int(len(food101_subset) * 0.9)
    test_i = len(food101_subset) - traning_i
    f_train, f_test = random_split(food101_subset, [traning_i, test_i])
    return f_train, f_test

f_train, f_test = load_data()
print(len(f_train), len(f_test))

transform done
iter:  0
set()
iter:  1000
{54, 23}
iter:  2000
{85, 54, 23}
iter:  3000
{86, 85, 54, 23}
iter:  4000
{77, 23, 85, 86, 54, 92}
iter:  5000
{73, 77, 23, 85, 86, 54, 92}
iter:  6000
{4, 73, 77, 23, 85, 86, 54, 92}
iter:  7000
{4, 39, 73, 77, 48, 23, 85, 86, 54, 92}
iter:  8000
{96, 4, 39, 73, 77, 48, 23, 85, 86, 54, 92}
iter:  9000
{96, 4, 70, 39, 73, 77, 48, 23, 85, 86, 54, 92}
iter:  10000
{96, 4, 70, 39, 73, 77, 78, 48, 81, 23, 85, 86, 54, 92}
iter:  11000
{96, 4, 70, 39, 73, 77, 78, 48, 81, 80, 23, 85, 86, 54, 92}
iter:  12000
{96, 4, 70, 39, 7, 73, 77, 78, 48, 81, 80, 23, 85, 86, 54, 92}
iter:  13000
{96, 0, 4, 70, 39, 7, 73, 6, 77, 78, 48, 81, 80, 23, 85, 86, 54, 92}
iter:  14000
{0, 4, 6, 7, 23, 26, 39, 48, 54, 70, 73, 77, 78, 80, 81, 85, 86, 92, 96}
iter:  15000
{0, 4, 6, 7, 23, 26, 39, 48, 54, 70, 73, 77, 78, 80, 81, 84, 85, 86, 92, 96}
len:  15001
13500 1501


In [19]:
t.cuda.is_available()
print(len(f_train))

13500


In [20]:
print(len(f_train))

13500
